In [158]:
import pandas as pd
import numpy as np
import string
from collections import Counter

from sklearn.metrics import accuracy_score, classification_report

from sklearn.utils.class_weight import compute_class_weight

from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('name_gender.csv')
df.head()

,name,gender
0,Aaban,M
1,Aabha,F
2,Aabid,M
3,Aabriella,F
4,Aada,F


In [196]:
df.shape

(95026, 2)

In [3]:
letters = list(string.ascii_lowercase)

letters.extend([i+b for i in letters for b in letters])

print(letters[:26])

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [194]:
all_letters = []
new_cols = []
for name in df.name:
    
    name_list = list(name.lower())
    first_letter = name_list[0]
    last_letter = name_list[-1]
    length_of_name = len(name_list)
    i=0
    while i < length_of_name-1:
        name_list.append(name_list[i]+name_list[i+1])
        i+=1
        
    results = dict(Counter(name_list))
    results['first_letter_'+first_letter] = 1
    results['last_letter_'+last_letter] = 1
    results['length_of_name'] = length_of_name
    new_cols.append(results)
    all_letters = list(set(all_letters+name_list+['first_letter_'+first_letter,'last_letter_'+last_letter]))
    #print(name,results)
    
print(all_letters[:10])
print(len(all_letters))
    

['bl', 'q', 'x', 'ec', 'gp', 'cb', 'ly', 'dl', 'first_letter_x', 'sq']
696


In [195]:
new_cols[:5]

[{'a': 3,
  'b': 1,
  'n': 1,
  'aa': 1,
  'ab': 1,
  'ba': 1,
  'an': 1,
  'first_letter_a': 1,
  'last_letter_n': 1,
  'length_of_name': 5},
 {'a': 3,
  'b': 1,
  'h': 1,
  'aa': 1,
  'ab': 1,
  'bh': 1,
  'ha': 1,
  'first_letter_a': 1,
  'last_letter_a': 1,
  'length_of_name': 5},
 {'a': 2,
  'b': 1,
  'i': 1,
  'd': 1,
  'aa': 1,
  'ab': 1,
  'bi': 1,
  'id': 1,
  'first_letter_a': 1,
  'last_letter_d': 1,
  'length_of_name': 5},
 {'a': 3,
  'b': 1,
  'r': 1,
  'i': 1,
  'e': 1,
  'l': 2,
  'aa': 1,
  'ab': 1,
  'br': 1,
  'ri': 1,
  'ie': 1,
  'el': 1,
  'll': 1,
  'la': 1,
  'first_letter_a': 1,
  'last_letter_a': 1,
  'length_of_name': 9},
 {'a': 3,
  'd': 1,
  'aa': 1,
  'ad': 1,
  'da': 1,
  'first_letter_a': 1,
  'last_letter_a': 1,
  'length_of_name': 4}]

In [6]:
additional_cols = pd.DataFrame(new_cols)

additional_cols

In [8]:
new_df = pd.concat([df,additional_cols],axis=1)

new_df.head()

,name,gender,a,b,n,aa,ab,ba,an,first_letter_a,...,tq,first_letter_u,first_letter_v,first_letter_w,fg,first_letter_x,xh,first_letter_y,first_letter_z,zx
0,Aaban,M,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aabha,F,3.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aabid,M,2.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aabriella,F,3.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aada,F,3.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
additional_cols_first_last = [col for col in additional_cols.columns if col[:5]=='first' or col[:4]=='last']
additional_cols_letters = [col for col in additional_cols.columns if col[:5]!='first' and col[:4]!='last' and col[:4]!='leng']

new_df = new_df[list(df.columns)+['length_of_name']+additional_cols_first_last+additional_cols_letters]
new_df = new_df.fillna(0)
new_df.head()

,name,gender,length_of_name,first_letter_a,last_letter_n,last_letter_a,last_letter_d,last_letter_m,last_letter_h,last_letter_v,...,pk,fw,qq,qv,tk,fp,tq,fg,xh,zx
0,Aaban,M,5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aabha,F,5,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aabid,M,5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aabriella,F,9,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aada,F,4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X = new_df[new_df.columns[2:]].values
y = new_df[new_df.columns[1]].values
print(X[:10])
print(y[:10])


[[5. 1. 1. ... 0. 0. 0.]
 [5. 1. 0. ... 0. 0. 0.]
 [5. 1. 0. ... 0. 0. 0.]
 ...
 [7. 1. 0. ... 0. 0. 0.]
 [5. 1. 1. ... 0. 0. 0.]
 [6. 1. 0. ... 0. 0. 0.]]
['M' 'F' 'M' 'F' 'F' 'M' 'M' 'M' 'M' 'M']


In [141]:
train_idx, test_idx = train_test_split(new_df.index, test_size=0.33, random_state=42)

print(train_idx[:10], test_idx[:10])

X_train, X_test, y_train, y_test = X[train_idx],X[test_idx], y[train_idx], y[test_idx]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Int64Index([60453, 34683, 87756, 2242, 25815, 61171, 36048, 57675, 5120,
            40516],
           dtype='int64') Int64Index([3030, 88133, 93041, 77162, 21789, 70650, 3241, 21297, 40127,
            25305],
           dtype='int64')
(63667, 697) (31359, 697) (63667,) (31359,)


In [142]:
print(Counter(y_test))


Counter({'F': 19847, 'M': 11512})


In [143]:
clf = RandomForestClassifier(n_estimators=71,random_state=42)
clf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=71, random_state=42)

In [144]:
sorted(list(zip(additional_cols.columns,clf.feature_importances_)), key=lambda x: x[1],reverse=True)

[('aa', 0.10875559194768414),
 ('n', 0.038003496039403104),
 ('a', 0.03120038249602499),
 ('ik', 0.027313569397739264),
 ('ha', 0.016564868315192407),
 ('z', 0.016089805942378767),
 ('it', 0.01604447272405649),
 ('is', 0.01361664359407104),
 ('last_letter_t', 0.013316934749393734),
 ('last_letter_a', 0.012369830066835301),
 ('y', 0.012184955901194065),
 ('d', 0.011972542043660255),
 ('e', 0.011292470337912844),
 ('h', 0.011228326644966351),
 ('iy', 0.011151811453803371),
 ('t', 0.010784808769723936),
 ('ea', 0.009777199180162396),
 ('za', 0.00954780307185255),
 ('ab', 0.008755423719888335),
 ('last_letter_n', 0.008247242881510953),
 ('f', 0.008194724381444514),
 ('et', 0.00803535386253966),
 ('o', 0.007812096290899195),
 ('ae', 0.007766992514815367),
 ('il', 0.007582656204162081),
 ('j', 0.0075355109825194925),
 ('dy', 0.0071769370066711346),
 ('q', 0.0071693784544560575),
 ('ai', 0.007110708791018937),
 ('le', 0.006853735501591611),
 ('last_letter_s', 0.006770855342050192),
 ('an', 0.

In [145]:


y_pred = clf.predict(X_test)

print(accuracy_score(y_test,y_pred))

print(classification_report(y_test,y_pred))

0.9088618897286266
              precision    recall  f1-score   support

           F       0.92      0.93      0.93     19847
           M       0.88      0.87      0.87     11512

    accuracy                           0.91     31359
   macro avg       0.90      0.90      0.90     31359
weighted avg       0.91      0.91      0.91     31359



In [152]:
training_names = list(new_df.iloc[train_idx]['name'])

training_names[:10]

['Mazarine',
 'Idesha',
 'Triya',
 'Ajan',
 'Ednamae',
 'Melynie',
 'Jaavon',
 'Malay',
 'Analynn',
 'Jerald']

In [180]:
def format_name(name):
    name_list = list(name.lower())
    first_letter = name_list[0]
    last_letter = name_list[-1]
    length_of_name = len(name_list)
    i=0
    while i < length_of_name-1:
        name_list.append(name_list[i]+name_list[i+1])
        i+=1
        
    results = dict(Counter(name_list))
    results['first_letter_'+first_letter] = 1
    results['last_letter_'+last_letter] = 1
    results['length_of_name'] = length_of_name
    prediction_input = []
    fields = new_df.columns[2:]
    for field in fields:
        if field in results.keys():
            prediction_input.append(results[field])
        else:
            prediction_input.append(0.0)
            
    return [prediction_input]

def predict_name(name):
    
    output_pred = clf.predict(format_name(name))[0]
    
    pred_confidence = round(np.max(clf.predict_proba(format_name(name)))*100)
    
    name_found = 'FOUND' if name in training_names else 'NOT FOUND' 
    
    decode = {'M':'Male',
             'F':'Female'}
    
    print('The name "{0}" is {1}% likely to be {2}. The name was {3} in the training data.\n'.format(name,pred_confidence,decode[output_pred],name_found))
    

In [199]:
predict_name('Beatriz')
predict_name('Bellatrix')
predict_name('Rodrigo')
predict_name('Roderick')
predict_name('Guilherme')
predict_name('Guilhermina')
predict_name('Joceli')
predict_name('Julia')
predict_name('Juliana')
predict_name('Julie')

The name "Beatriz" is 97% likely to be Female. The name was FOUND in the training data.

The name "Bellatrix" is 76% likely to be Female. The name was NOT FOUND in the training data.

The name "Rodrigo" is 97% likely to be Male. The name was FOUND in the training data.

The name "Roderick" is 97% likely to be Male. The name was NOT FOUND in the training data.

The name "Guilherme" is 73% likely to be Male. The name was FOUND in the training data.

The name "Guilhermina" is 92% likely to be Female. The name was NOT FOUND in the training data.

The name "Joceli" is 66% likely to be Female. The name was NOT FOUND in the training data.

The name "Julia" is 96% likely to be Female. The name was NOT FOUND in the training data.

The name "Juliana" is 100% likely to be Female. The name was NOT FOUND in the training data.

The name "Julie" is 89% likely to be Female. The name was FOUND in the training data.

